# Module 05: Understanding Webhooks and Triggers

**Difficulty**: ⭐⭐  
**Estimated Time**: 50 minutes  
**Prerequisites**: 
- Module 04: Initial Configuration and First Workflow
- Understanding of basic HTTP concepts
- n8n running and accessible

## Learning Objectives

By the end of this notebook, you will be able to:

1. Explain what webhooks are and how they differ from polling
2. Create workflows with different trigger types (schedule, webhook, manual)
3. Test webhooks locally using n8n's --tunnel feature
4. Implement webhook security with authentication tokens
5. Build real-world webhook examples for common services

## What Are Webhooks?

### Traditional Polling vs Webhooks

**Polling** (the old way):
- Your app asks "Anything new?" every X minutes
- Wastes resources checking when nothing happened
- Delayed response (only checks every X minutes)
- Like checking your mailbox every hour

**Webhooks** (the modern way):
- External service tells you "Something happened!"
- Only activates when there's actually new data
- Instant response (milliseconds, not minutes)
- Like the mailman ringing your doorbell

### How Webhooks Work

1. **You provide a URL** to the external service (e.g., GitHub, Stripe)
2. **Service sends HTTP POST** to that URL when events occur
3. **Your workflow receives data** and processes it
4. **You respond** with HTTP 200 OK

### Why Webhooks Matter for n8n

Webhooks enable **real-time automation**:
- New email arrives → Trigger workflow
- Payment received → Send confirmation  
- Form submitted → Add to database
- File uploaded → Process and notify

Without webhooks, you'd need to check constantly (polling) which wastes resources.

In [ ]:
# Polling vs Webhook comparison

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# Comparison table
comparison = {
    'Aspect': [
        'Response Time',
        'Resource Usage',
        'Complexity',
        'Reliability',
        'Scalability',
        'Setup Difficulty'
    ],
    'Polling': [
        'Delayed (check interval)',
        'High (constant checking)',
        'Simple',
        'Can miss events',
        'Poor (more checks = more cost)',
        'Easy'
    ],
    'Webhooks': [
        'Instant (real-time)',
        'Low (event-driven)',
        'Moderate',
        'No missed events',
        'Excellent (scales with events)',
        'Moderate'
    ]
}

df = pd.DataFrame(comparison)

print("=" * 80)
print("POLLING VS WEBHOOKS COMPARISON")
print("=" * 80)
print()
print(df.to_string(index=False))
print()
print("=" * 80)
print()
print("💡 When to use each:")
print("   Polling: Service doesn't support webhooks, or very infrequent checks needed")
print("   Webhooks: Real-time response needed, high-volume events, production systems")

## n8n Trigger Types

### 1. Manual Trigger

- **When**: Click "Execute Workflow" button
- **Use case**: Testing, one-off tasks
- **No activation needed**: Works without Active toggle

### 2. Schedule Trigger

- **When**: Time-based (every hour, daily, weekly, cron)
- **Use case**: Reports, backups, periodic checks
- **Requires**: Workflow must be Active
- **Example**: Send daily sales report at 9 AM

### 3. Webhook Trigger

- **When**: HTTP request received at webhook URL
- **Use case**: Real-time integrations, form submissions
- **Requires**: Workflow must be Active, URL must be accessible
- **Example**: Process new Stripe payment

### 4. Polling Trigger (Service-specific)

- **When**: Regular checks to external service
- **Use case**: Services without webhook support
- **Requires**: Workflow Active, credentials configured
- **Example**: Check Gmail for new emails every 5 minutes

### 5. Error Trigger

- **When**: Another workflow fails
- **Use case**: Error notifications, recovery workflows
- **Requires**: Workflow Active
- **Example**: Send alert when payment workflow fails

In [ ]:
# Trigger types reference

trigger_types = {
    'Trigger Type': [
        'Manual',
        'Schedule',
        'Webhook',
        'Polling (Gmail, etc)',
        'Error Trigger'
    ],
    'Activation Required': [
        'No',
        'Yes',
        'Yes',
        'Yes',
        'Yes'
    ],
    'Real-time': [
        'N/A',
        'No',
        'Yes',
        'No',
        'Yes'
    ],
    'Resource Usage': [
        'Minimal',
        'Low',
        'Minimal',
        'Medium',
        'Minimal'
    ],
    'Best For': [
        'Testing',
        'Time-based tasks',
        'Real-time events',
        'Services w/o webhooks',
        'Error handling'
    ]
}

triggers_df = pd.DataFrame(trigger_types)

print("=" * 90)
print("N8N TRIGGER TYPES REFERENCE")
print("=" * 90)
print()
print(triggers_df.to_string(index=False))
print()
print("=" * 90)

## Creating a Schedule Trigger Workflow

Let's create a workflow that runs every day at 9 AM.

### Step-by-Step

1. **Create new workflow**
2. **Add Schedule Trigger node**:
   - Click "+" → Search "Schedule Trigger"
   - Select it
3. **Configure schedule**:
   - Mode: "Every day"
   - Hour: 9
   - Minute: 0
4. **Add action nodes** (e.g., HTTP Request to get data)
5. **Activate workflow** (toggle Active ON)

### Important: Timezone

Schedule triggers use the timezone set in:
- Environment variable: `GENERIC_TIMEZONE`
- Default: UTC (which may not be your local time!)

**Always set your timezone correctly** to avoid workflows running at wrong times.

### Cron Expression (Advanced)

For complex schedules, use cron:
- `0 9 * * *` = 9:00 AM daily
- `0 9 * * 1` = 9:00 AM every Monday
- `*/15 * * * *` = Every 15 minutes
- `0 0 1 * *` = Midnight on 1st of each month

## Creating a Webhook Trigger Workflow

Webhooks are the most powerful trigger type. Let's build one.

### Step-by-Step

1. **Create new workflow**
2. **Add Webhook node**:
   - Click "+" → Search "Webhook"
   - Select it
3. **Configure webhook**:
   - HTTP Method: POST (most common)
   - Path: Give it a name (e.g., "form-submission")
   - Response Mode: "Immediately"
4. **Note the webhook URL**:
   - Production URL: `http://localhost:5678/webhook/form-submission`
   - Test URL: Shown when you click "Listen for Test Event"
5. **Add processing nodes** (Set, Code, etc.)
6. **Activate workflow**

### Testing the Webhook

Use PowerShell to send a test request:

```powershell
$body = @{
    name = "John Doe"
    email = "john@example.com"
    message = "Hello from webhook!"
} | ConvertTo-Json

Invoke-RestMethod -Uri "http://localhost:5678/webhook/form-submission" `
    -Method Post `
    -Body $body `
    -ContentType "application/json"
```

You should see the workflow execute!

In [ ]:
# Webhook URL structure helper

def generate_webhook_url(base_url="http://localhost:5678", webhook_path="my-webhook"):
    """
    Generate n8n webhook URLs for different scenarios.
    """
    
    print("=" * 70)
    print("N8N WEBHOOK URL STRUCTURE")
    print("=" * 70)
    print()
    
    # Production webhook
    prod_url = f"{base_url}/webhook/{webhook_path}"
    print(f"Production Webhook URL:")
    print(f"  {prod_url}")
    print()
    print("  When to use: Workflow is Active, for real automation")
    print()
    
    # Test webhook
    test_url = f"{base_url}/webhook-test/{webhook_path}"
    print(f"Test Webhook URL:")
    print(f"  {test_url}")
    print()
    print("  When to use: Testing during development, workflow can be inactive")
    print()
    
    print("=" * 70)
    print()
    print("💡 Testing with curl (from Command Prompt/PowerShell):")
    print()
    print(f'curl -X POST {prod_url} ^')
    print('  -H "Content-Type: application/json" ^')
    print('  -d "{\\"test\\": \\"data\\"}"')
    print()
    print("=" * 70)

# Example
generate_webhook_url(
    base_url="http://localhost:5678",
    webhook_path="contact-form"
)

## Testing Webhooks with --tunnel

### The Problem

External services (GitHub, Stripe, etc.) can't reach `http://localhost:5678` because:
- Localhost is private to your computer
- Your router/firewall blocks incoming connections
- You don't have a public IP address

### The Solution: --tunnel

n8n's `--tunnel` flag creates a temporary public URL.

**For npm:**
```bash
n8n start --tunnel
```

**For Docker:**
```powershell
docker run -d --name n8n -p 5678:5678 -v n8n_data:/home/node/.n8n docker.n8n.io/n8nio/n8n start --tunnel
```

### What You'll See

```
Tunnel URL: https://random-name-12345.n8n.cloud
```

This URL is accessible from anywhere on the internet!

### Using the Tunnel URL

In your webhook node, n8n automatically uses the tunnel URL when:
- You click "Listen for Test Event"
- Workflow is active and --tunnel is running

### Important Warnings

🚨 **DEVELOPMENT ONLY**
- Tunnel URL changes every restart
- Connection can be unstable
- Data passes through n8n's servers (privacy concern)
- Not suitable for production

For production webhooks, you need:
- Static public IP or domain name
- SSL certificate (HTTPS)
- Proper firewall configuration

## Webhook Security

### The Risk

Unprotected webhooks let **anyone** trigger your workflow:
- Spam attacks consuming resources
- Malicious data injection
- Unauthorized workflow execution

### Security Methods

#### 1. Obscure Webhook Path

Instead of `/webhook/contact-form`, use:
- `/webhook/contact-form-a8f3d9b2c4e1`

**Pros**: Simple, no configuration
**Cons**: Security through obscurity, not true security

#### 2. Header Authentication

Require a secret token in HTTP header:

```json
{
  "X-Webhook-Token": "your-secret-token-here"
}
```

In n8n:
1. Add IF node after webhook
2. Check: `{{ $json.headers["x-webhook-token"] }} equals your-secret-token-here`
3. True path: Process data
4. False path: Return error

#### 3. Signature Verification (Best)

Services like GitHub, Stripe send cryptographic signatures:
- They hash the payload with a secret
- You verify the signature
- Prevents tampering

n8n has built-in support for many services.

#### 4. IP Whitelisting

Only accept webhooks from known IPs:
- Configure Windows Firewall rules
- Or check IP in workflow IF node

**Best for**: Services with published IP ranges (GitHub, Stripe, etc.)

In [ ]:
# Security levels comparison

security_comparison = {
    'Method': [
        'No security',
        'Obscure path',
        'Header token',
        'Signature verification',
        'IP whitelist + signature'
    ],
    'Security Level': [
        '❌ None',
        '⚠️ Weak',
        '✅ Good',
        '✅✅ Strong',
        '✅✅✅ Maximum'
    ],
    'Setup Difficulty': [
        'None',
        'Trivial',
        'Easy',
        'Moderate',
        'Hard'
    ],
    'Recommended For': [
        'Never',
        'Internal testing only',
        'Low-risk workflows',
        'Production webhooks',
        'High-security production'
    ]
}

security_df = pd.DataFrame(security_comparison)

print("=" * 100)
print("WEBHOOK SECURITY METHODS COMPARISON")
print("=" * 100)
print()
print(security_df.to_string(index=False))
print()
print("=" * 100)
print()
print("⚠️  NEVER use webhooks without security in production!")
print("   Minimum: Use header token authentication")
print("   Best: Use signature verification when available")

## Real-World Webhook Examples

### Example 1: GitHub Repository Events

**Use case**: Get notified when someone stars your repo

**Setup**:
1. Create webhook workflow in n8n
2. Start n8n with --tunnel
3. In GitHub repo: Settings → Webhooks → Add webhook
4. Payload URL: Your tunnel URL
5. Content type: application/json
6. Events: "Star"
7. Process the webhook data in n8n

### Example 2: Form Submissions

**Use case**: Process contact form from your website

**HTML form**:
```html
<form id="contact">
  <input name="name" required>
  <input name="email" type="email" required>
  <textarea name="message" required></textarea>
  <button>Submit</button>
</form>

<script>
document.getElementById('contact').onsubmit = async (e) => {
  e.preventDefault();
  const formData = new FormData(e.target);
  const data = Object.fromEntries(formData);
  
  await fetch('http://localhost:5678/webhook/contact', {
    method: 'POST',
    headers: {'Content-Type': 'application/json'},
    body: JSON.stringify(data)
  });
  
  alert('Thank you! Message received.');
};
</script>
```

**n8n workflow**:
1. Webhook trigger
2. Set node to format data
3. Email node to send notification
4. Database node to store submission

### Example 3: Stripe Payment Webhooks

**Use case**: Send confirmation email when payment succeeds

**Setup**:
1. Stripe Dashboard → Developers → Webhooks
2. Add endpoint with your n8n webhook URL
3. Select events: `payment_intent.succeeded`
4. Copy webhook signing secret
5. In n8n, verify signature before processing

## Exercises

### Exercise 1: Create a Schedule Workflow (Easy)

Create a workflow that:
1. Uses Schedule Trigger set to every 30 minutes
2. Makes HTTP GET to `https://api.coindesk.com/v1/bpi/currentprice.json`
3. Extracts the USD price
4. Logs it (Code node with `console.log()`)
5. Activate the workflow

Wait 30 minutes and check Executions to verify it ran.

### Exercise 2: Build a Secure Webhook (Medium)

Create a webhook workflow with header authentication:

1. Add Webhook node (POST method)
2. Add IF node to check header `X-API-Key` equals `my-secret-key-12345`
3. True path: Set node extracting data, return success message
4. False path: Return error message
5. Test both paths using PowerShell/curl

**Test with valid key:**
```powershell
Invoke-RestMethod -Uri "http://localhost:5678/webhook/secure" `
    -Method Post `
    -Headers @{"X-API-Key"="my-secret-key-12345"} `
    -Body '{"data": "test"}' `
    -ContentType "application/json"
```

**Test with invalid key:**
```powershell
Invoke-RestMethod -Uri "http://localhost:5678/webhook/secure" `
    -Method Post `
    -Headers @{"X-API-Key"="wrong-key"} `
    -Body '{"data": "test"}' `
    -ContentType "application/json"
```

### Exercise 3: Multi-Trigger Workflow (Hard)

Create a workflow that can be triggered THREE ways:

1. **Webhook** → Process form submission
2. **Schedule** → Run cleanup task daily
3. **Manual** → Test execution

Workflow should:
- Detect which trigger activated it (check `$json` from trigger)
- Use Switch node to route to different processing
- Each path does something different

**Hint**: Multiple trigger nodes in one workflow is possible!

## Summary

In this module, you learned:

### Webhook Fundamentals
1. **Webhooks vs Polling**: Real-time vs delayed, resource-efficient vs wasteful
2. **How Webhooks Work**: External service → HTTP POST → Your workflow
3. **Webhook URLs**: Production vs Test URLs

### Trigger Types
4. **Manual**: Testing and one-off tasks
5. **Schedule**: Time-based automation (cron expressions)
6. **Webhook**: Real-time event-driven automation
7. **Polling**: For services without webhook support
8. **Error**: Workflow error handling

### Testing & Security
9. **--tunnel Flag**: Temporary public URL for testing
10. **Security Methods**: Obscure paths, header auth, signatures, IP whitelisting
11. **Real-World Examples**: GitHub, forms, Stripe payments

### Key Takeaways

- ✅ Webhooks enable real-time automation
- ✅ Always secure webhooks in production (minimum: header authentication)
- ✅ --tunnel for development only, never production
- ✅ Set correct timezone for schedule triggers
- ✅ Webhook path should be unpredictable (add random string)

### Best Practices

1. **Test with --tunnel first**: Before configuring external services
2. **Implement security**: Never expose unprotected webhooks
3. **Log webhook data**: Helps debug issues
4. **Respond quickly**: External services timeout after 30 seconds usually
5. **Validate input**: Always check received data structure

### What's Next

- **Module 06**: Security best practices for production webhooks
- **Module 07**: Performance optimization for high-volume webhooks
- **Module 10**: Production deployment strategies with public URLs

### Additional Resources

- [n8n Webhook Documentation](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.webhook/)
- [Cron Expression Generator](https://crontab.guru/)
- [Webhook.site](https://webhook.site/) - Test webhook receiver
- [RequestBin](https://requestbin.com/) - Inspect webhook payloads